In [ ]:
import pandas as pd
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
def extract_number(filename):
    match = re.search(r'(\d+)\.(xls|xlsx)$', filename)
    return int(match.group(1)) if match else float('inf')

# 2023 https://www.shfe.com.cn/reports/tradedata/datadownload/historicaldata/futures/202401/P020240119587353273027.zip
# 2024 https://www.shfe.com.cn/reports/tradedata/datadownload/historicaldata/futures/202501/P020250103380580546440.zip

zip_paths = [
    'P020240119587353273027.zip', 
    'P020250103380580546440.zip'
]

all_dfs = []
for zip_path in zip_paths:
    with zipfile.ZipFile('C:/Users/wayne/Downloads/' + zip_path) as z:
        file_list = []
        for info in z.infolist():
            filename = (info.filename.encode('cp437').decode('gbk'))
            if '所内合约行情报表' in filename and not filename.endswith('/'):
                file_list.append((filename, info))
    
        file_list.sort(key=lambda x: extract_number(x[0]))
        for filename, info in file_list:
            with z.open(info.filename) as f:
                try: 
                    all_dfs.append(pd.read_excel(f, skiprows=[0,1,2], header=0))
                except Exception as e:
                    print(filename)

def pre(df):
    df = df.copy()
    df = df[df['Date'].notna()]
    df['Date'] = df['Date'].astype(int)
    df.loc[:, 'Contract'] = df['Contract'].ffill()
    return df
    
df = pd.concat([pre(t) for t in all_dfs], ignore_index=True)
df = df.drop(columns='Unnamed: 14')

In [ ]:
df.shape

In [ ]:
filtered = df[
    (df['Contract'].str.match(r'^au\d{4}$'))
]
filtered.loc[:, 'Date'] = pd.to_datetime(filtered['Date'].astype(str), format='%Y%m%d')


sns.set(style="whitegrid")
# 创建一个画布
plt.figure(figsize=(24, 10))

# 使用 seaborn.lineplot 绘图，hue 表示不同合约用不同颜色
sns.lineplot(data=filtered, x='Date', y='Close', hue='Contract', marker='o', lw=1, ms=4)

# 设置标题和标签
plt.title('Close Price Over Time by Contract (AU Series)', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Close Price', fontsize=12)

# 避免重叠日期标签
plt.xticks(rotation=45)
plt.legend(title='Contract')

# 展示图表
plt.tight_layout()
plt.show()

In [ ]:
df = filtered


top2 = df.groupby('Date', group_keys=False).apply(lambda g: g.nlargest(2, 'Volume')).reset_index(drop=True)

top2['rank'] = top2.groupby((top2['Date']).ne(top2['Date'].shift()).cumsum()).cumcount()

pivot = top2.pivot(index='Date', columns='rank', values=['Contract', 'Close', 'Volume'])

spread_df = pd.DataFrame({
    'Date': pivot.index,
    'Contract_1': pivot[('Contract', 0)],
    'Contract_2': pivot[('Contract', 1)],
    'Close_1': pivot[('Close', 0)],
    'Close_2': pivot[('Close', 1)],
    'Spread': pivot[('Close', 0)] - pivot[('Close', 1)],
})


In [ ]:
spread_df['Spread'] = pd.to_numeric(spread_df['Spread'], errors='coerce')
spread_df[spread_df['Spread'].abs() > 2.5]

spread_df[(spread_df['Date'] > datetime(2024,5,1)) & (spread_df['Date'] < datetime(2024,6,1)) ]

In [ ]:
spread_df['Spread_abs'] = spread_df['Spread'].abs()
sns.set(style="whitegrid")

# 创建画布
plt.figure(figsize=(12, 6))

# 绘制 Spread 曲线
sns.lineplot(
    data=spread_df,
    x='Date',
    y='Spread_abs',
    marker='o',    # 可选：显示数据点
    lw=1.5         # 线条粗细
)

# 添加标题和标签
plt.title('Contract Spread Over Time (by Volume)', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Spread (Close Difference)', fontsize=12)

# 旋转日期标签，防止重叠
plt.xticks(rotation=45)

# 自动调整布局
plt.tight_layout()

# 显示图表
plt.show()

In [ ]:
from itertools import combinations
max_spreads = []

for date, group in df.groupby('Date'):
    # 确保至少有两个合约才能比较
    if len(group) < 2:
        continue

    contracts = group['Contract'].tolist()
    closes = group.set_index('Contract')['Close']

    max_diff = 0
    pair = ('', '')

    # 两两组合比较
    for c1, c2 in combinations(contracts, 2):
        diff = abs(closes[c1] - closes[c2])
        if diff > max_diff:
            max_diff = diff
            pair = (c1, c2)

    max_spreads.append({
        'Date': date,
        'Contract_1': pair[0],
        'Contract_2': pair[1],
        'Max_Spread': max_diff
    })

# 转为 DataFrame
max_spread_df = pd.DataFrame(max_spreads)

sns.set(style="whitegrid")

# 创建画布
plt.figure(figsize=(12, 6))

# 绘制 Spread 曲线
sns.lineplot(
    data=max_spread_df,
    x='Date',
    y='Max_Spread',
    marker='o',    # 可选：显示数据点
    lw=1.5         # 线条粗细
)

# 添加标题和标签
plt.title('Contract Spread Over Time (by Volume)', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Spread (Close Difference)', fontsize=12)

# 旋转日期标签，防止重叠
plt.xticks(rotation=45)

# 自动调整布局
plt.tight_layout()

# 显示图表
# plt.show()

max_spread_df[max_spread_df['Max_Spread'] > 19]